![Azure AI Search](../assets/ai_search.png)

# Microsoft Semantic Kernel + Azure AI Search

Vamos a crear un ejemplo de aplicación RAG utilizando Semantic Kernel y Azure AI Search

In [9]:
// Install the DotNetEnv package using the NuGet package manager
#r "nuget: dotenv.net, 3.2.1"
#r "nuget: Microsoft.SemanticKernel, 1.31.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.31.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.31.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.31.0-preview"

#r "nuget: PdfPig, 0.1.9"

Installed Packages dotenv.net, 3.2.1 Microsoft.SemanticKernel, 1.31.0 Microsoft.SemanticKernel.Agents.Core, 1.31.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.31.0-alpha Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.31.0-preview PdfPig, 0.1.9

In [16]:
using System.ComponentModel;
using System.Text.Json;
using System.Threading;

using dotenv.net;

using Azure;

using Microsoft.Extensions.VectorData;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Libraries for agents from Semantic Kernel
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.History;
using Microsoft.SemanticKernel.Agents.OpenAI;

using UglyToad.PdfPig;

// Load the .env file
DotEnv.Load();

In [13]:

// Access the environment variables
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var azureOpenAIModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_MODEL_NAME"); 
var azureOpenAIDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME");
var azureOpenAIApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION");
var azureOpenAIEmbeddingsModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME");
var azureOpenAIEmbeddingsDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME");

var azureAISearchEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_SEARCH_ENDPOINT");
var azureAISearchKey = Environment.GetEnvironmentVariable("AZURE_AI_SEARCH_KEY");



In [17]:
internal sealed class TextSnippet<TKey>
{
    [VectorStoreRecordKey]
    public required TKey Key { get; set; }

    [VectorStoreRecordData]
    public string? Text { get; set; }

    [VectorStoreRecordData]
    public string? ReferenceDescription { get; set; }

    [VectorStoreRecordData]
    public string? ReferenceLink { get; set; }

    [VectorStoreRecordVector(1536)]
    public ReadOnlyMemory<float> TextEmbedding { get; set; }
}

In [18]:
#pragma warning disable SKEXP0010

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(azureOpenAIDeployment, azureOpenAIEndpoint, azureOpenAIKey)
       .AddAzureOpenAITextEmbeddingGeneration(azureOpenAIEmbeddingsDeployment, azureOpenAIEndpoint, azureOpenAIKey)
       .AddAzureAISearchVectorStoreRecordCollection<TextSnippet<string>>("documents", new Uri(azureAISearchEndpoint), new AzureKeyCredential(azureAISearchKey))
       ;

var kernel = builder.Build();